<a href="https://colab.research.google.com/github/siddhantj802/EmailSender/blob/main/grouping_nums.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=e3448e68e137b1dfc3e0d8c6ff532fd4f359b13be230d13cbefa2c20b8ab60a6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [4]:
# Import a Spark function from library
from pyspark.sql.functions import col

In [5]:
data = [56, 89, 101, 178, 235, 64, 156, 275, 89, 112, 199, 287, 33, 201, 78, 44]
df = spark.createDataFrame([(x,) for x in data], ["number"])


In [6]:
df.show()

+------+
|number|
+------+
|    56|
|    89|
|   101|
|   178|
|   235|
|    64|
|   156|
|   275|
|    89|
|   112|
|   199|
|   287|
|    33|
|   201|
|    78|
|    44|
+------+



In [18]:
from pyspark.sql.functions import floor

lower_bound = 1
upper_bound = 100
group_label = 1

df = df.withColumn("group", floor((df.number - lower_bound) / upper_bound) + group_label)


In [8]:
df.show()

+------+-----+
|number|group|
+------+-----+
|    56|    1|
|    89|    1|
|   101|    2|
|   178|    2|
|   235|    3|
|    64|    1|
|   156|    2|
|   275|    3|
|    89|    1|
|   112|    2|
|   199|    2|
|   287|    3|
|    33|    1|
|   201|    3|
|    78|    1|
|    44|    1|
+------+-----+



In [60]:
grouped_df = df.groupBy("group").agg({"number": "collect_list"})

In [56]:
grouped_df.show(truncate = False)

+-----+----------------------------+
|group|collect_list(number)        |
+-----+----------------------------+
|1    |[56, 89, 64, 89, 33, 78, 44]|
|3    |[235, 275, 287, 201]        |
|2    |[101, 178, 156, 112, 199]   |
+-----+----------------------------+



In [62]:
#change column name to nums

grouped_df = grouped_df.select(col("group").alias("Group"), col("collect_list(number)").alias("nums"))

In [63]:
from pyspark.sql.functions import col, expr

grouped_df = grouped_df.select(col("Group"), expr("transform(nums, x -> cast(x as int))").alias("nums"))


In [65]:
from pyspark.sql.functions import expr

# Calculate the sum of numbers group-wise
sum_df = grouped_df.withColumn("sum_numbers", expr("aggregate((nums), 0, (acc, x) -> acc + x)"))

# Find the group with the highest sum
max_sum_group = sum_df.orderBy("sum_numbers", ascending=False).first()

# Extract the highest sum and group information
highest_sum = max_sum_group["sum_numbers"]
group_with_highest_sum = max_sum_group["Group"]

# Print the highest sum and group
print("Highest Sum:", highest_sum)
print("Group with Highest Sum:", group_with_highest_sum)


Highest Sum: 998
Group with Highest Sum: 3
